In [1]:
# @title Prepare Environment
import torch
import os
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install torch-scatter
!pip install torch-geometric
!pip install nystrom-attention
!pip install tensorboard
!git clone https://github.com/andrea-grandi/bio_project
!cd bio_project

2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:0000:01
Cloning into 'bio_project'...
remote: Enumerating objects: 8461, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 8461 (delta 157), reused 320 (delta 125), pack-reused 8087 (from 1)
Receiving objects: 100% (8461/8461), 588.01 MiB | 46.88 MiB/s, done.
Resolving deltas: 100% (825/825), done.
Updating files: 100% (2379/2379), done.


In [2]:
# @title Load Libraries
from torch_geometric.data import Dataset
import glob
from torch_geometric.data import data
from torch_geometric.loader import DataLoader
import sys
sys.path.append("/kaggle/working/bio_project/src/bio_project")
import argparse
import wandb
from torch.nn import BCEWithLogitsLoss
import torch
import os
import wandb
from argparse import Namespace
import time
import tqdm

In [3]:
data_root="/kaggle/input/embeddings-final/final_embeddings/processed"

In [4]:
# @title Dataset
class CustomDataset(Dataset):
    def __init__(self,root,data_type):
        self.path=os.path.join(root,data_type,"*")
        self.data=glob.glob(self.path)
        self.slides=[torch.load(self.data[idx]) for idx in range(len(self.data))]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.slides[idx]
        return sample
        
train_dataset=CustomDataset(data_root,"train")
test_dataset=CustomDataset(data_root,"test")
train_loader=DataLoader(train_dataset,batch_size=1,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=1,shuffle=True)

<ipython-input-4-d7b681704474>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.slides=[torch.load(self.data[idx]) for idx in range(len(self.data))]


In [5]:
from utils.test import test
from models import selectModel

In [6]:
# @title General Parser
def get_args():
    parser = argparse.ArgumentParser(description='Train Buffermil')

    # Optimization arguments
    group1 = parser.add_argument_group("optimization")
    group1.add_argument('--lr', default=0.00001,
                        type=float, help='learning rate')
    group1.add_argument('--weight_decay', default=0.00001,
                        type=float, help='Weight decay [5e-3]')

    # GNN arguments
    group2 = parser.add_argument_group("gnn")
    group2.add_argument('--residual', default=False, action="store_true",)
    group2.add_argument('--num_layers', default=1, type=int,
                        help='number of Graph layers')
    group2.add_argument('--dropout', default=True, action="store_true")
    group2.add_argument('--dropout_rate', default=0.2, type=float)
    group2.add_argument('--layer_name', default="GAT",
                        type=str, help='layer graph name')
    group2.add_argument('--heads', default=3, type=int,
                        help='layer graph name')

    # Training arguments
    group3 = parser.add_argument_group("training")
    group3.add_argument('--seed', default=12, type=int,
                        help='seed for reproducibility')
    group3.add_argument('--n_epoch', default=500,
                        type=int, help='number of epochs')

    # Dimensions arguments
    group4 = parser.add_argument_group("dimensions")
    group4.add_argument('--n_classes', default=1, type=int,
                        help='Number of output classes [2]')
    group4.add_argument('--c_hidden', default=256,
                        type=int, help='intermediate size ')
    group4.add_argument('--input_size', default=384,
                        type=int, help='input size ')

    # Dataset arguments
    group5 = parser.add_argument_group("dataset")
    group5.add_argument('--scale', default="0", type=str,
                        help='scale resolution')
    group5.add_argument('--dataset', default="cam", type=str,
                        choices=["cam", "lung"], help='input size ')
    group5.add_argument('--datasetpath',  type=str, help='dataset path')

    # Distillation arguments
    group6 = parser.add_argument_group("distillation")
    group6.add_argument('--lamb', default=1, type=float, help='lambda')
    group6.add_argument('--beta', default=1, type=float, help='beta')
    group6.add_argument('--temperature', default=1.5, type=float, help='temperature')
    group6.add_argument('--add_bias', default=True, action="store_true")
    group6.add_argument('--max', default=True, action="store_true")
    group6.add_argument('--checkpoint', default=None, type=str, help='checkpoint')

    parser.add_argument('--tag', default="split", type=str, help='train strategy')
    parser.add_argument('--modeltype', default="Buffermil", type=str, help='train strategy')
    parser.add_argument('--project', default="bio_project", type=str, help='project name for wandb')
    parser.add_argument('--model', default="bio_project", type=str, help='project name for wandb')
    parser.add_argument('--wandbname', default="main", type=str, help='project name for wandb')

    group7 = parser.add_argument_group("submitit")
    group7.add_argument('--partition', default="prod",type=str,help='partition name')
    group7.add_argument('--time', default=120, type=float, help='job duration')
    group7.add_argument('--nodes', default=1, type=int, help='number of jobs')
    group7.add_argument('--job_name', default="dasmil",type=str,help="job name")
    group7.add_argument('--mem', default=32, type=int, help='ram requested GB')
    group7.add_argument('--job_parallel', default=10, type=int, help='number of jobs in parallel')
    group7.add_argument('--logfolder', default="logfolder", type=str, help='log folder location name')

    # Buffermil parameters
    group8 = parser.add_argument_group("submitit")
    group8.add_argument("--randomstore", default=False, help="ramdom sampling during the buffer storage")
    group8.add_argument("--bufferaggregate", default="mean", choices=["mean","max","diffmax"], help="type of buffer aggregation")
    group8.add_argument("--ntop", default=10, help="number of patches stored in the buffer per each image")
    group8.add_argument('--buffer_freq', default=10, type=int, help='frequency to update the buffer')
    
    filtered_args = [arg for arg in sys.argv if not arg.startswith('-f')]
    args = parser.parse_args(filtered_args[2:])
    return args

In [7]:
args = get_args()
print(args)

Namespace(lr=1e-05, weight_decay=1e-05, residual=False, num_layers=1, dropout=True, dropout_rate=0.2, layer_name='GAT', heads=3, seed=12, n_epoch=500, n_classes=1, c_hidden=256, input_size=384, scale='0', dataset='cam', datasetpath=None, lamb=1, beta=1, temperature=1.5, add_bias=True, max=True, checkpoint=None, tag='split', modeltype='Buffermil', project='bio_project', model='bio_project', wandbname='main', partition='prod', time=120, nodes=1, job_name='dasmil', mem=32, job_parallel=10, logfolder='logfolder', randomstore=False, bufferaggregate='mean', ntop=10, buffer_freq=10)


In [ ]:
# @title Select Model
import ipywidgets as widgets
from IPython.display import display

# Crea un menu a discesa
dropdown = widgets.Dropdown(
    options=["Buffermil", "CustomBuffermil"],
    value='Buffermil',
    description='Choose:',
)

# Mostra il widget
display(dropdown)

# Recupera il valore selezionato
def on_value_change(change):
    print(f"Selected: {change['new']}")

dropdown.observe(on_value_change, names='value')
args.modeltype = dropdown.value

Dropdown(description='Choose:', options=('Buffermil', 'CustomBuffermil'), value='Buffermil')

In [10]:
model = selectModel(args)
model.kl = None

model Buffermil
error loading state dict
error loading
Number of parameters: 345730
Memory usage: 0.001383936 GB


In [11]:
print(model)

Buffermil(
  (mil): MILNetBuffer(
    (i_classifier): FCLayer(
      (fc): Sequential(
        (0): Linear(in_features=384, out_features=1, bias=True)
      )
    )
    (b_classifier): BClassifierBuffer(
      (lin): Sequential(
        (0): Linear(in_features=384, out_features=384, bias=True)
        (1): ReLU()
      )
      (q): Sequential(
        (0): Linear(in_features=384, out_features=128, bias=True)
        (1): Tanh()
      )
      (v): Sequential(
        (0): Dropout(p=0.0, inplace=False)
        (1): Linear(in_features=384, out_features=384, bias=True)
      )
      (fcc): Conv1d(1, 1, kernel_size=(384,), stride=(1,))
    )
  )
)


In [12]:
# @title Training Method
def train(model,train_loader,test_loader,args):
    # Initialize wandb run
    run = wandb.init(project="bio_project_v4", name=args.modeltype)
    epochs = args.n_epoch
    wd = args.weight_decay
    lr = args.lr
    model.train()
    model = model.cuda()
    loss_module_instance = BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    # Test the initial model
    with torch.no_grad():
        start_test = time.time()
        metrics = test(model, testloader=test_loader)
        end_test = time.time()
        avg_score_higher_test, avg_score_lower_test, auc_value_higher_test, auc_value_lower_test, predictions, _, labels = metrics

        wandb.log({
            "acc_higher_test": avg_score_higher_test,
            "acc_lower_test": avg_score_lower_test,
            "auc_higher_test": auc_value_higher_test,
            "epoch": -1,
            "lr": scheduler.get_last_lr()[0]
        })
        
    BestPerformance = 0
    # Start training
    for idx,epoch in tqdm.tqdm(enumerate(range(epochs)), desc="epochs"):
        start_training = time.time()
        if hasattr(model, "preloop"):
            model.preloop(epoch, train_loader)
        # Iterate over the training data
        for _, data in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            data = data.cuda()
            x, edge_index, childof, level = data.x, data.edge_index, data.childof, data.level
            
            # Check if additional edge indices are present
            if data.__contains__("edge_index_2") and data.__contains__("edge_index_3"):
                edge_index2, edge_index3 = data.edge_index_2, data.edge_index_3
            else:
                edge_index2 = None
                edge_index3 = None

            try:
                results = model(x, edge_index, level, childof, edge_index2, edge_index3)
            except:
                continue
                
            bag_label = data.y.float()
            loss = model.compute_loss(loss_module_instance, results, bag_label)
            wandb.log({"loss": loss})
            loss.backward()
            optimizer.step()
            
        end_training = time.time()
        scheduler.step()
        start_test = time.time()
        metrics = test(model, testloader=test_loader)
        end_test = time.time()
        avg_score_higher_test, avg_score_lower_test, auc_value_higher_test, auc_value_lower_test, predictions, _, labels = metrics

        wandb.log({
            "acc_higher_test": avg_score_higher_test,
            "acc_lower_test": avg_score_lower_test,
            "auc_higher_test": auc_value_higher_test,
            "epoch": epoch,
            "lr": scheduler.get_last_lr()[0]
        })

    wandb.finish()

In [14]:
# @title Start Training
train(model, train_loader, test_loader, args)

epochs: 500it [00:51,  9.73it/s]


acc_higher_test,▁███████████████████████████████████████
acc_lower_test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auc_higher_test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
loss,▅▄█▄█▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,██████████▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
acc_higher_test,0.75
acc_lower_test,0
auc_higher_test,0.66667
epoch,499
loss,0.00725


In [13]:
wandb.login(key="d350c1926691a1f8b36ff558c9a28e9425f84fde")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: andrea-grandi (andrea-grandi-unimore). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True